# Notas Fiscais Eletrônicas da União - Portal da Transparência CGU NFe

In [ ]:
#| hide: true
! rm -rf ../data/inputs/cgu_nfe/*

## Fonte dos dados

O Portal da Transparência do Governo Federal disponibiliza dados sobre Notas Fiscais Eletrônicas (NFE) da União que podem ser baixados no endereço:

https://portaldatransparencia.gov.br/download-de-dados/notas-fiscais/

Cada mês possui um arquivo zipado que contém os dados das notas fiscais eletrônicas emitidas no período. O nome do arquivo segue o padrão `YYYYMM_NFe.zip`, onde `YYYY` é o ano e `MM` é o mês.

Dentro do Zip encontramos os seguintes arquivos em formato CSV e codificação `latin-1`:
- `YYYYMM_NFe_NotaFiscal.csv`: Contém os dados das notas fiscais eletrônicas.
- `YYYYMM_NFe_NotaFiscalItem.csv`: Contém os dados dos itens das notas fiscais eletrônicas.
- `YYYYMM_NFe_NotaFiscalEvento.csv`: Contém os dados dos eventos atribuidos às notas fiscais eletrônicas.

Para baixar os dados de um mês específico, é preciso baixar da url:

https://portaldatransparencia.gov.br/download-de-dados/notas-fiscais/[YYYYMM]


No portal da transparência, há dados de Novembro de 2019 até o mês atual com exceção do período de janeiro de 2020 até abril de 2021. Não há nada que indique se os dados do mês atual são atualizados diariamente. 



## Baixando os dados

Vamos baixar os dados 2022 a 2024. Mas primeiro, um teste.

In [ ]:
import io, zipfile, pandas as pd
from pathlib import Path
from urllib import request


def cgu_download(year, month):
    out_dir = Path("../data/inputs/cgu_nfe/"); out_dir.mkdir(parents=True, exist_ok=True)
    url = f"https://portaldatransparencia.gov.br/download-de-dados/notas-fiscais/{year}{month:02d}"
    with request.urlopen(url) as response:
        buf = io.BytesIO(response.read())
    with zipfile.ZipFile(buf) as z:
        [(out_dir / Path(n).name).write_bytes(z.read(n)) for n in z.namelist() if n.lower().endswith(".csv")]

In [ ]:
cgu_download(2022, 1)

In [ ]:
import os
os.listdir("../data/inputs/cgu_nfe/")

Ok, um download funcionou.  Vamos baixar todos em paralelo.

In [ ]:
import asyncio, nest_asyncio, tqdm.asyncio as tqdma
nest_asyncio.apply() # Jupyter notebook async compatibility


for year in range(2022, 2025): # max of 12 concurrent downloads
    tasks = [asyncio.to_thread(cgu_download, year, month) for month in range(1, 13)]
    await tqdma.tqdm.gather(*tasks, desc=f"Downloading {year}", total=len(tasks))

In [ ]:
os.listdir("../data/inputs/cgu_nfe/")[:9]

## Usando DuckDB para manipular os dados 


O primeiro passo é criar um arquivo duckdb com as extensões necessárias para manipular os dados.

In [ ]:
import duckdb
def create_db(p, ext_core=("spatial","httpfs","json","excel"), ext_community=("zipfs",)):
    c = duckdb.connect(p)
    for e in ext_core: c.install_extension(e); c.load_extension(e)
    for e in ext_community: c.install_extension(e, repository="community"); c.load_extension(e)
    return c

db = create_db("../data/outputs/cgu/nfe.db")
db.execute("PRAGMA version").df()

Note que estamos usando versão 1.3.2 do DuckDB.

### RAW.NFE__NOTAS

In [ ]:
### CGU.NOTAS

In [ ]:
db.execute("""
CREATE SCHEMA IF NOT EXISTS raw;

CREATE OR REPLACE TABLE raw.nfe__notas AS
SELECT *
FROM read_csv(
    '../data/inputs/cgu_nfe/*Fiscal.csv',
    delim=';',
    header=true,
    encoding='latin-1',
    escape='\"',
    auto_type_candidates=['VARCHAR']
);
""").df()

Criamos uma tabela `raw.nfe__notas` com 5 milhões de linhas.

In [ ]:
db.execute("describe raw.nfe__notas").df()


O dicionário de dados com as colunas de cada arquivo pode ser encontrado em:

https://portaldatransparencia.gov.br/dicionario-de-dados/notas-fiscais

In [ ]:
db.query("from raw.nfe__notas").df().head(3)

In [ ]:
db.execute("summarize raw.nfe__notas").df()

MODELO é sempre o mesmo. "DATA EMISSÃO" e "DATA/HORA EVENTO MAIS RECENTE" são do tipo `timestamp` e "VALOR NOTA FISCAL" é do tipo `decimal`.  Assumindo que nenhum valor ultrapassa 999 bilhões, podemos usar decimal(14,2).

In [ ]:
db.execute("SELECT COUNT(*) AS n_rows FROM raw.nfe__notas").df()


A tabela de notas tem aproximadamente 5.2 milhões de linhas.  Mas algumas são repetidas. 

In [ ]:
db.execute("""
           SELECT "CHAVE DE ACESSO", COUNT(*) AS n_rows 
           FROM raw.nfe__notas group by "CHAVE DE ACESSO" 
           order by n_rows desc""").df()

In [ ]:
db.execute("""       
    SELECT "DATA EMISSÃO", "EVENTO MAIS RECENTE", "DATA/HORA EVENTO MAIS RECENTE", "VALOR NOTA FISCAL" from raw.nfe__notas where "CHAVE DE ACESSO" = '21240706049258000169550010002625451000741910'
    """).df()

In [ ]:
db.execute("""       
    SELECT DISTINCT * from raw.nfe__notas where "CHAVE DE ACESSO" = '21240706049258000169550010002625451000741910'
    """).df()

In [ ]:
distinct_notas = db.execute("SELECT DISTINCT * FROM raw.nfe__notas").df()
distinct_notas.shape

In [ ]:
db.execute("""SELECT SUM((REPLACE("VALOR NOTA FISCAL", ',', '.')::DECIMAL(14,2))) as valor_nota FROM distinct_notas""").df()

100 mil linhas, aproximadamente, são repetidas. E o valor total das notas é aproximadamente 232 bilhões de reais.

## Normalizando os dados

In [ ]:

from IPython.display import display
import numpy as np
from decimal import Decimal

pd.set_option("display.float_format", lambda x: f"{x:_.2f}") 
def underscore_fmt(x):
    if pd.isna(x): return ""
    if isinstance(x, (int, np.integer)):   return f"{x:_d}"
    if isinstance(x, (float, np.floating)): return f"{x:_.0f}"
    if isinstance(x, Decimal):             return f"{x:_.0f}"
    return x



In [ ]:
cols = db.execute("""
SELECT column_name
FROM (summarize raw.nfe__notas)
WHERE approx_unique < 30
""").df()["column_name"].tolist()
cols

In [ ]:

qid = lambda c: '"' + c.replace('"', '""') + '"'

# build UNION of per-column distinct lists
sql = " UNION ALL ".join(
    f"""
    SELECT 
        '{c}' AS column_name, 
        len(LIST(DISTINCT {qid(c)})) AS n_distinct,
        LIST_SORT(LIST(DISTINCT {qid(c)})) AS distinct_values
    FROM raw.nfe__notas
    """
    for c in cols
)


enum_df = db.execute(sql).df()

enum_df

In [ ]:
list(enum_df.distinct_values)

In [ ]:
create_enums_sql = """
DROP TYPE IF EXISTS tipo_modelo;
CREATE TYPE tipo_modelo AS ENUM ('55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 OU 1A');

DROP TYPE IF EXISTS tipo_evento;
CREATE TYPE tipo_evento AS ENUM (
    'Autorização de Uso', 
    'Cancelamento da NF-e', 
    'Carta de correção',
    'Manifestação do destinatário - Ciência da operação',
    'Manifestação do destinatário - Confirmação da operação',
    'Manifestação do destinatário - Desconhecimento da operação',
    'Manifestação do destinatário - Operação não realizada',
    'Sem informação'
);

DROP TYPE IF EXISTS uf;
CREATE TYPE uf AS ENUM (
    'AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO',
    'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR',
    'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO'
);

DROP TYPE IF EXISTS tipo_contribuinte;
CREATE TYPE tipo_contribuinte AS ENUM (
    'CONTRIBUINTE ICMS',
    'CONTRIBUINTE ISENTO', 
    'NÃO CONTRIBUINTE'
);

DROP TYPE IF EXISTS tipo_destino;
CREATE TYPE tipo_destino AS ENUM (
    '1 - OPERAÇÃO INTERNA',
    '2 - OPERAÇÃO INTERESTADUAL',
    '3 - OPERAÇÃO COM EXTERIOR'
);

DROP TYPE IF EXISTS tipo_consumidor;
CREATE TYPE tipo_consumidor AS ENUM (
    '0 - NORMAL',
    '1 - CONSUMIDOR FINAL'
);

DROP TYPE IF EXISTS tipo_presenca;
CREATE TYPE tipo_presenca AS ENUM (
    '0 - NÃO SE APLICA',
    '1 - OPERAÇÃO PRESENCIAL',
    '2 - OPERAÇÃO NÃO PRESENCIAL, PELA INTERNET',
    '3 - OPERAÇÃO NÃO PRESENCIAL, TELEATENDIMENTO',
    '5 - NÃO INFORMADO',
    '9 - OPERAÇÃO NÃO PRESENCIAL, OUTROS'
);
"""

In [ ]:
db.execute("CREATE SCHEMA IF NOT EXISTS cgu;")
db.execute(create_enums_sql)


In [ ]:
db.execute("select column_name from (describe raw.nfe__notas)").df()["column_name"].to_list()

In [ ]:
create_cgu__notas_sql = """
SELECT DISTINCT
  "CHAVE DE ACESSO" AS chave_acesso, -- identificador único da nota fiscal eletrônica
  "MODELO"::tipo_modelo AS modelo, -- código do modelo de documento fiscal (sempre 55 - NFE)
  "SÉRIE"::USMALLINT AS serie, -- série da nota fiscal. juntamente com o número identifica unicamente a nota fiscal
  "NÚMERO"::BIGINT AS numero, -- número da nota fiscal. juntamente com a série identifica unicamente a nota fiscal
  "NATUREZA DA OPERAÇÃO" AS natureza_operacao, -- descrição da natureza da operação. campo de preenchimento livre.
  strptime("DATA EMISSÃO", '%d/%m/%Y %H:%M:%S') AS timestamp_emissao, -- data e hora de emissão da nota fiscal
  timestamp_emissao::date AS data_emissao, -- data de emissão da nota fiscal
  month(data_emissao)::INT AS mes_emissao, -- mês de emissão da nota fiscal
  year(data_emissao)::INT AS ano_emissao, -- ano de emissão da nota fiscal
  "EVENTO MAIS RECENTE"::tipo_evento AS ultimo_evento, -- evento mais recente associado a nota fiscal (indica status atual da nota)
  strptime("DATA/HORA EVENTO MAIS RECENTE", '%d/%m/%Y %H:%M:%S') AS timestamp_ultimo_evento, -- data e hora da última atualização do status da nota fiscal
  "CPF/CNPJ Emitente" AS emitente, -- cpf ou cnpj emissor da nota fiscal. fornecedor.
  "RAZÃO SOCIAL EMITENTE" AS nome_emitente, -- razão social do emissor da nota fiscal
  "INSCRIÇÃO ESTADUAL EMITENTE"::BIGINT AS inscricao_estadual_emitente, -- inscrição estadual do emissor da nota fiscal
  "UF EMITENTE"::uf AS uf_emitente, -- sigla unidade federativa do emissor da nota fiscal
  "MUNICÍPIO EMITENTE" AS municipio_emitente, -- nome município do emissor da nota fiscal. 
  "CÓDIGO ÓRGÃO SUPERIOR DESTINATÁRIO"::INT as codigo_orgao_superior_destinatario, -- código do órgão superior destinatário da nota fiscal
  "ÓRGÃO SUPERIOR DESTINATÁRIO" AS orgao_superior_destinatario, -- código do órgão superior destinatário da nota fiscal
  "CÓDIGO ÓRGÃO DESTINATÁRIO"::INT AS codigo_orgao_destinatario, -- código do órgão destinatário da nota fiscal
  "ÓRGÃO DESTINATÁRIO" AS orgao_destinatario,
  "CNPJ DESTINATÁRIO" AS destinatario, -- cnpj do destinatário da nota fiscal. cliente.
  "NOME DESTINATÁRIO" AS nome_destinatario, -- nome do destinatário da nota fiscal
  "UF DESTINATÁRIO"::uf AS uf_destinatario, -- sigla da unidade federativa do destinatário da nota fiscal
  "INDICADOR IE DESTINATÁRIO"::tipo_contribuinte AS indicador_ie_destinatario, -- indicador de inscrição estadual do destinatário da nota fiscal
  "DESTINO DA OPERAÇÃO"::tipo_destino AS destino_operacao, -- destino da operação da nota fiscal (interna, interestadual, exterior)
  "CONSUMIDOR FINAL"::tipo_consumidor AS indicador_consumidor_final, -- indica se o destinatário é consumidor final (0 - normal, 1 - consumidor final)
  "PRESENÇA DO COMPRADOR"::tipo_presenca AS indicador_presenca_comprador, -- indica se o comprador estava presente na operação (0 - não se aplica, 1 - presencial, 2 - não presencial, 3 - teleatendimento, 5 - não informado, 9 - outros)  
  REPLACE("VALOR NOTA FISCAL", ',', '.')::decimal(14,2) AS valor_nota_fiscal -- valor total da nota fiscal
FROM raw.nfe__notas order by timestamp_emissao; 
"""

In [ ]:
db.execute("CREATE OR REPLACE TABLE cgu.notas AS " + create_cgu__notas_sql).df()

In [ ]:
db.execute("from cgu.notas limit 2").df()

In [ ]:
db.execute("select sum(valor_nota_fiscal) as sum_decimal, sum(valor_nota_fiscal::float) as sum_float from cgu.notas").df()

É importante usar decimal para evitar problemas de precisão com valores monetários.

#### Usando 2024 para checar os dados

In [ ]:
db.execute("select sum(valor_nota_fiscal) from cgu.notas where ano_emissao = 2024").df()

Esse valor ainda não bate com o relatório do Portal da Transparência, que diz que há 1.7 milhões de notas e um valor de 72 bilhões de reais.

![](./portal_2024.png)

In [ ]:
df = db.execute("FROM cgu.notas where ano_emissao = 2024").df()
df.shape

Mas a quantidade de notas é bem próxima. Provavelmente estamos contando notas que foram canceladas. O estranho é que considerando então menos notas, o valor total deveria ser ainda menor.

In [ ]:
db.execute("""
SELECT numero, serie, emitente, destinatario, COUNT(*) AS chaves
FROM cgu.notas where ano_emissao = 2024
GROUP BY ALL
HAVING chaves > 1
ORDER BY chaves DESC
""").df()

In [ ]:
1_631_584 - 1_631_490

In [ ]:
dedupl = db.execute("""
SELECT *
FROM (
  SELECT
    chave_acesso, numero, serie, emitente, destinatario,
    timestamp_emissao, ultimo_evento, timestamp_ultimo_evento,
    ROW_NUMBER() OVER (
      PARTITION BY numero, serie, emitente, destinatario
      ORDER BY timestamp_emissao DESC, timestamp_ultimo_evento DESC
    ) AS rn
  FROM cgu.notas
)
WHERE rn = 1
ORDER BY numero, serie, emitente, destinatario, timestamp_emissao DESC, timestamp_ultimo_evento DESC
""").df()
dedupl

In [ ]:
dedupl_sql = """
ALTER TABLE cgu.notas
ADD COLUMN IF NOT EXISTS duplicada BOOLEAN DEFAULT FALSE;

UPDATE cgu.notas AS n
SET duplicada = TRUE
FROM (
  SELECT
    chave_acesso,
    ROW_NUMBER() OVER (
      PARTITION BY numero, serie, emitente, destinatario
      ORDER BY timestamp_emissao DESC, timestamp_ultimo_evento DESC
    ) AS rn
  FROM cgu.notas
) d
WHERE n.chave_acesso = d.chave_acesso
  AND d.rn > 1;
"""
db.execute(dedupl_sql)


In [ ]:
db.query("SELECT COUNT(*) AS valid_rows FROM cgu.notas WHERE duplicada=FALSE AND ano_emissao = 2024").df()

In [ ]:
db.query("SELECT sum(valor_nota_fiscal) FROM cgu.notas WHERE duplicada=FALSE AND ano_emissao = 2024").df()

Agora há 1_631_342 notas válidas para 2024, com um total de aproximadamente 70 bilhões de reais.
Um pouco diferente do que o Portal da Transparência diz, mas bem próximo (1_631_490 e 72 bilhões de reais).
Vamos ver se os valores fazem sentido.

In [ ]:
db.query("SELECT uf_destinatario, sum(valor_nota_fiscal) FROM cgu.notas WHERE uf_destinatario in ('PB', 'SP') and duplicada=FALSE AND ano_emissao = 2024 group by ALL").df()

In [ ]:
db.query("SELECT ultimo_evento, count(*) as n  FROM cgu.notas WHERE duplicada=FALSE AND ano_emissao = 2024 group by all").df()

In [ ]:
db.execute("""
SELECT chave_acesso, numero, serie, emitente, timestamp_emissao, ultimo_evento, timestamp_ultimo_evento, valor_nota_fiscal
FROM cgu.notas
WHERE ultimo_evento = 'Cancelamento da NF-e'
""").df()

In [ ]:
db.execute("""
SELECT *
FROM cgu.notas where ultimo_evento = 'Cancelamento da NF-e'
and ano_emissao = 2024
""").df()

Só há cancelamentos em 2022 e 2023. Em 2024 não há cancelamentos.

In [ ]:
db.execute("""
SELECT chave_acesso, numero, serie, emitente, timestamp_emissao, ultimo_evento, timestamp_ultimo_evento, valor_nota_fiscal
FROM cgu.notas
WHERE (numero, serie, emitente) IN (
  SELECT numero, serie, emitente
  FROM cgu.notas
  GROUP BY ALL
  HAVING COUNT(*) > 1
)
ORDER BY numero asc, serie asc, emitente asc, timestamp_emissao desc
""").df()

In [ ]:
db.execute("""
SELECT chave_acesso, numero, serie, emitente, destinatario, timestamp_emissao,ultimo_evento, valor_nota_fiscal
FROM cgu.notas
WHERE (numero, serie, emitente, destinatario) IN (
    SELECT numero, serie, emitente, destinatario
    FROM cgu.notas where ano_emissao = 2024
    GROUP BY ALL
    HAVING COUNT(*) > 1
    
)
ORDER BY numero asc, serie asc, emitente asc, timestamp_emissao desc
""").df()

#### Usando 2023

![](./portal2023.png)

In [ ]:
display(db.execute("""
SELECT count(*)
FROM cgu.notas 
where ano_emissao = 2023
""").df().style.format(underscore_fmt))

In [ ]:
db.execute("""
SELECT sum(valor_nota_fiscal) as valor_total
FROM cgu.notas 
where ano_emissao = 2023
""").df()

In [ ]:
display(db.execute("""
SELECT count(*)
FROM cgu.notas 
where ano_emissao = 2023 and ultimo_evento != 'Cancelamento da NF-e'
""").df().style.format(underscore_fmt))

In [ ]:
db.execute("""
SELECT sum(valor_nota_fiscal) as valor_total
FROM cgu.notas 
where ano_emissao = 2023 and ultimo_evento != 'Cancelamento da NF-e'
""").df()

In [ ]:
display(db.execute("""
SELECT count(*)
FROM cgu.notas 
where ano_emissao = 2023 and duplicada=FALSE
""").df().style.format(underscore_fmt))

In [ ]:
display(db.execute("""
SELECT count(*)
FROM cgu.notas 
where ano_emissao = 2023 and duplicada=FALSE and ultimo_evento != 'Cancelamento da NF-e'
""").df().style.format(underscore_fmt))

In [ ]:
db.execute("""
SELECT sum(valor_nota_fiscal) as valor_total
FROM cgu.notas 
where ano_emissao = 2023 and duplicada=FALSE and ultimo_evento != 'Cancelamento da NF-e'
""").df()

### Checando participação da Saúde

In [ ]:
db.execute(""" SELECT distinct orgao_superior_destinatario, codigo_orgao_superior_destinatario FROM cgu.notas order by codigo_orgao_superior_destinatario""").df()

In [ ]:
db.execute("""
SELECT ano_emissao, sum(valor_nota_fiscal) as valor_total
FROM cgu.notas 
where duplicada=FALSE and ultimo_evento != 'Cancelamento da NF-e' and codigo_orgao_superior_destinatario = 36000
group by ano_emissao
""").df()

In [ ]:
db.execute("""
SELECT 
    ano_emissao,
    SUM(CASE WHEN codigo_orgao_superior_destinatario = 36000 THEN valor_nota_fiscal ELSE 0 END) AS valor_saude,
    SUM(valor_nota_fiscal) AS valor_total,
    100.0 * SUM(CASE WHEN codigo_orgao_superior_destinatario = 36000 THEN valor_nota_fiscal ELSE 0 END) 
          / SUM(valor_nota_fiscal) AS pct
FROM cgu.notas 
WHERE duplicada = FALSE 
  AND ultimo_evento != 'Cancelamento da NF-e'
GROUP BY ano_emissao
ORDER BY ano_emissao
""").df()

### RAW.NFE__ITENS

In [ ]:
db.execute("""
CREATE OR REPLACE TABLE raw.nfe__itens AS
SELECT *
FROM read_csv(
    '../data/inputs/cgu_nfe/*Item.csv',
    delim=';',
    header=true,
    encoding='latin-1',
    escape='\"',
    auto_type_candidates=['VARCHAR']
);
""").df()

In [ ]:
num_itens = db.execute(""" SELECT COUNT(*) FROM raw.nfe__itens """).df()
num_itens.style.format(underscore_fmt)


In [ ]:
db.execute("DESCRIBE raw.nfe__itens").df()

In [ ]:
db.query("SELECT sum(valor_nota_fiscal) FROM cgu.notas WHERE duplicada=FALSE AND ano_emissao = 2024").df()

In [ ]:
db.execute(""" SELECT sum((REPLACE("VALOR TOTAL", ',', '.')::decimal(14,2))) as valor_total FROM raw.nfe__itens where "CHAVE DE ACESSO" in (select chave_acesso from cgu.notas where ano_emissao = 2024 and duplicada = FALSE)""").df()

Valores deveriam bater

In [ ]:
db.execute(""" SELECT sum((REPLACE("VALOR TOTAL", ',', '.')::decimal(14,2))) as valor_total FROM raw.nfe__itens where "CHAVE DE ACESSO" in (select chave_acesso from cgu.notas where ano_emissao = 2023 and duplicada = FALSE and ultimo_evento!= 'Cancelamento da NF-e')""").df()

In [ ]:
db.execute("""SELECT "CHAVE DE ACESSO" from raw.nfe__itens where "CHAVE DE ACESSO" not in (select chave_acesso from cgu.notas)""").df()

Ok, todas as chaves de acesso da tabela de itens estão na tabela de notas.

In [ ]:
db.execute("select column_name from (describe raw.nfe__itens)").df()["column_name"].to_list()

In [ ]:
itens_sql = """
SELECT DISTINCT
  "CHAVE DE ACESSO" AS chave_acesso,
  strptime("DATA EMISSÃO", '%d/%m/%Y %H:%M:%S') AS timestamp_emissao,
  "CÓDIGO ÓRGÃO SUPERIOR DESTINATÁRIO"::INT AS codigo_orgao_superior_destinatario, -- código do órgão superior destinatário da nota fiscal
  "CÓDIGO ÓRGÃO DESTINATÁRIO"::INT AS codigo_orgao_destinatario, -- código do órgão destinatário da nota fiscal
  timestamp_emissao::date AS data_emissao, -- data de emissão da nota fiscal
  month(data_emissao)::INT AS mes_emissao, -- mês de emissão da nota fiscal
  year(data_emissao)::INT AS ano_emissao, -- ano de emissão da nota fiscal
  "NÚMERO PRODUTO"::SMALLINT AS numero_produto,
  "DESCRIÇÃO DO PRODUTO/SERVIÇO" AS descricao_produto,
  "CÓDIGO NCM/SH" AS ncm,
  "NCM/SH (TIPO DE PRODUTO)" AS descricao_ncm,
  "CFOP"::SMALLINT AS cfop,
  REPLACE("QUANTIDADE", ',', '.')::DECIMAL(14,2) AS quantidade,
  "UNIDADE" AS unidade,
  REPLACE("VALOR UNITÁRIO", ',', '.')::DECIMAL(14,2) AS valor_unitario,
  REPLACE("VALOR TOTAL", ',', '.')::DECIMAL(14,2) AS valor_item
FROM raw.nfe__itens order by chave_acesso, numero_produto
"""

db.execute(f"CREATE OR REPLACE TABLE cgu.itens AS {itens_sql}").df()

In [ ]:
db.execute(""" FROM cgu.itens limit 3""").df()

In [ ]:
db.execute("""SELECT ano_emissao, sum(valor_item) FROM cgu.itens group by ano_emissao order by ano_emissao""").df()

### RAW.NFE__EVENTOS

In [ ]:
db.execute("""
CREATE OR REPLACE TABLE raw.nfe__eventos AS
SELECT *
FROM read_csv(
    '../data/inputs/cgu_nfe/*Evento.csv',
    delim=';',
    header=true,
    encoding='latin-1',
    escape='\"',
    auto_type_candidates=['VARCHAR']
);
""").df()

In [ ]:
num_eventos = db.execute(""" SELECT COUNT(*) FROM raw.nfe__eventos """).df()
num_eventos.style.format(underscore_fmt)

In [ ]:
num_eventos = db.execute(""" SELECT COUNT(*) FROM (SELECT DISTINCT * FROM raw.nfe__eventos) AS subquery """).df()
num_eventos.style.format(underscore_fmt)

In [ ]:
db.execute("summarize raw.nfe__eventos").df()

In [ ]:
eventos_sql = """
SELECT DISTINCT
    "CHAVE DE ACESSO" AS chave_acesso, -- identificador da nota fiscal em que este evento foi registrado. juntamente com timestamp_evento, forma a chave primária
    "SÉRIE" AS serie, -- série da nota fiscal. juntamente com o número identifica unicamente a nota fiscal de um emitente
    "NÚMERO" AS numero, -- número da nota fiscal. juntamente com a série identifica unicamente a nota fiscal de um emitente
    "NATUREZA DA OPERAÇÃO" AS natureza_operacao, -- descrição da natureza da operação. campo de preenchimento livre.
    strptime("DATA EMISSÃO", '%d/%m/%Y %H:%M:%S') AS timestamp_emissao, -- data e hora de emissão da nota fiscal
    "EVENTO" AS evento, -- tipo de evento registrado (autorização de uso, cancelamento, etc)
    strptime("DATA/HORA EVENTO", '%d/%m/%Y %H:%M:%S') AS timestamp_evento, -- data e hora do evento
    "DESCRIÇÃO EVENTO" AS descricao_evento, -- descrição do evento (Protocolo)
    "MOTIVO EVENTO" AS motivo_evento, -- motivo do evento. campo de preenchimento livre.
FROM raw.nfe__eventos order by chave_acesso, timestamp_evento desc; 
"""
num_eventos = db.execute("CREATE TABLE IF NOT EXISTS cgu.eventos AS " + eventos_sql).df()
num_eventos.style.format(underscore_fmt)

In [ ]:
eventos = db.execute("""FROM cgu.eventos""").df()
eventos

In [ ]:
db.execute("SELECT * FROM cgu.notas where chave_acesso = '28220137937325000105550010000003631472879112'").df()


In [ ]:
db.execute("SELECT * FROM cgu.eventos where emitente = '29369516000351'").df()